# Databricks Managed MCP Server for Genie, Vector Search, and Custom Functions

In [0]:
%pip install -U "mcp>=1.9" "databricks-sdk[openai]" "mlflow>=3.1.0" "databricks-agents>=1.0.0" "databricks-mcp" databricks_ai_bridge

dbutils.library.restartPython()

  Using cached databricks_sdk-0.58.0-py3-none-any.whl.metadata (39 kB)
INFO: pip is looking at multiple versions of databricks-agents to determine which version is compatible with other requirements. This could take a while.
  Using cached databricks_agents-1.1.0-py3-none-any.whl.metadata (3.5 kB)
  Using cached databricks_agents-1.0.1-py3-none-any.whl.metadata (3.6 kB)
  Using cached databricks_agents-1.0.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached mlflow_skinny-3.1.1-py3-none-any.whl.metadata (30 kB)
INFO: pip is still looking at multiple versions of databricks-agents to determine which version is compatible with other requirements. This could take a while.
  Using cached mlflow-3.1.1-py3-none-any.whl.metadata (29 kB)
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
  Using cached datab

## MCP with Genie

In [0]:
space_id = "01f05e591d711e81bd981dae9784abbd"

# Initialize the Databricks MCP client with the specified server URL and workspace client
mcp_client = DatabricksMCPClient(
    server_url=f"https:///adb-984752964297111.11.azuredatabricks.net/api/2.0/mcp/genie/{space_id}",
    workspace_client=workspace_client,
)

# Access the genie client from the MCP client
genie = mcp_client.client.genie

# Start a conversation with the genie and wait for a response
response = genie.start_conversation_and_wait(
    space_id=space_id,
    content="what are different type of order status?",
)

# Retrieve the query result from the message attachment
genie.get_message_attachment_query_result(
    space_id=space_id,
    message_id=response.message_id,
    conversation_id=response.conversation_id,
    attachment_id=response.attachments[0].attachment_id,
).statement_response.result.data_array,

([['cancelled'], ['pending'], ['processing'], ['completed']],)

## MCP with Databricks Vector Search

In [0]:
catalog_name = "mc"
schema_name = "fidel"

# Initialize the Databricks MCP client with the specified server URL and workspace client
mcp_client = DatabricksMCPClient(
    server_url=f"https:///adb-984752964297111.11.azuredatabricks.net/api/2.0/mcp/vector-search/{catalog_name}/{schema_name}",
    workspace_client=workspace_client,
)

# Query the vector search index with the specified index name and query text
mcp_client.client.vector_search_indexes.query_index(
    index_name="mc.fidel.vs_index",
    query_text="RAG",
    columns=["page_content_chunk"],
).result.data_array

[['.', 0.002849468],
 ['around Retrieval Augmented Generation (RAG), today, we’re excited to announce the public preview of Databricks Vector Search',
  0.0023659014],
 ['. We announced the private preview to a limited set of customers at the Data + AI Summit in June, which is now available to all our customers. Databricks Vector Search enables developers to improve the accuracy of their Retrieval Augmented Generation (RAG) and generative AI applications through similarity search over unstructured documents such as PDFs, Office Documents, Wikis, and more. Vector Search is part of the Databricks Data Intelligence Platform, making it easy for your RAG and Generative AI applications to use the proprietary data stored in your Lakehouse in a fast and secure manner and deliver accurate responses.',
  0.0018597686],
 ['What is vector search?\nVector search is a method used in information retrieval and Retrieval Augmented Generation (RAG) applications to find documents or records based on thei

## MCP with Custom Functions

In [0]:
import nest_asyncio
import pandas as pd
import json
from databricks_mcp import DatabricksMCPClient
from databricks.sdk import WorkspaceClient

workspace_client = WorkspaceClient()
mcp_server_url = (
    "https://adb-984752964297111.11.azuredatabricks.net/api/2.0/mcp/functions/system/ai"
)
nest_asyncio.apply()

code = """print("Hello, world!")"""


def test_connect_to_server(code):
    mcp_client = DatabricksMCPClient(
        server_url=mcp_server_url, workspace_client=workspace_client
    )

    tools = mcp_client.list_tools()
    print(f"Discovered tools {[t.name for t in tools]}")

    result = mcp_client.call_tool("system__ai__python_exec", {"code": code})
    return result.content[0].text


# display(pd.DataFrame([test_connect_to_server(code)]))

result = test_connect_to_server(code)

display(pd.DataFrame((json.loads(result))))